In [1]:
import os
import json
from dotenv import load_dotenv
from openai import OpenAI
import pandas as pd

load_dotenv("../.env")

client = OpenAI(api_key=os.environ['API_KEY'])

MODEL_NAME = "gpt-4o-mini"


In [2]:
# 住所から県名を推測
def extract_prefecture(address):
    prompt_text = f"次の住所に基づいて、対応する都道府県名を推測し、「都道府県名」のみを答えてください：{address}"

    try:
        response = client.chat.completions.create(
            model=MODEL_NAME,
            messages=[
                {"role": "user", "content": prompt_text},
            ],
            max_tokens=100,
            temperature=0.3
        )
        keywords = response.choices[0].message.content.strip()
        return keywords
    except Exception as e:
        print(f"API呼び出しでエラーが発生しました: {e}")
        return "エラー"

In [5]:
text = extract_prefecture("渋谷区道玄坂1丁目16-3")
print(text)

東京都


In [6]:
# ワークフロー
print("処理を開始します")

# ファイル読み込み
df = pd.read_excel('サンプルデータ.xlsx', sheet_name='住所')

# 住所列の内容から都道府県名を推測して都道府県列に格納
df['都道府県'] = df['住所'].apply(extract_prefecture)

# ファイルを保存
df.to_excel("都道府県付住所.xlsx", index=False)

print("Excelファイルに結果を保存しました")

処理を開始します
Excelファイルに結果を保存しました


In [7]:
# 電話番号フォーマット
def format_phone_numbers(phone_number):
    prompt_text = f"次の電話番号を統一した形式「xxx-xxxx-xxxx」に変換してください。電話番号のみを出力してください: {phone_number}"

    try:
        response = client.chat.completions.create(
            model=MODEL_NAME,
            messages=[
                {"role": "user", "content": prompt_text},
            ],
            max_tokens=100,
            temperature=0.3
        )

        keywords = response.choices[0].message.content.strip()
        return keywords
    except Exception as e:
        print(f"API呼び出しでエラーが発生しました: {e}")
        return "エラー"

In [8]:
text = format_phone_numbers("090.1234.5566")
print(text)

090-1234-5566


In [9]:
# ワークフロー
print("処理を開始します")

df = pd.read_excel('サンプルデータ.xlsx', sheet_name='電話番号')

df['フォーマット後'] = df['電話番号'].apply(format_phone_numbers)

df.to_excel("フォーマット後電話番号.xlsx", index=False)

print("Excelファイルに結果を保存しました。")

処理を開始します
Excelファイルに結果を保存しました。


In [10]:
from openai.types.chat import ChatCompletionToolParam

tools = [
    ChatCompletionToolParam(
        {
            "type": "function",
            "function": {
                "name": "get_products",
                "description": "グループごとに統一した商品名「group_name」と、グループに属する商品名のリスト「products」を結合した結果を返す",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "product_data": {
                            "type": "array",
                            "items": {
                                "product_name": {
                                    "type": "string",
                                    "description": "グループごとに統一した商品名",
                                },
                                "products": {
                                    "type": "string",
                                    "description": "グループに属する商品名をカンマ区切りで並べた文字列",
                                },
                            },
                        }
                    },
                    "required": ["product_name", "products"],
                },
            },
        }
    )
]

In [11]:
# ワークフロー
print("処理を開始します。")

df = pd.read_excel('サンプルデータ.xlsx', sheet_name='商品名')

product_data_text = df.astype(str)
prompt_text = f"""
次の商品名のリストから、似た名前の商品をグループ化し、グループごとに統一した商品名「product_name」を提案し、グループに属する商品名をカンマ区切りで並べた文字列「products」を作成し、「product_nameとproductsのリスト」で構成されるJSONデータ「product_data」を出力してください。
{product_data_text}
"""

response = client.chat.completions.create(
    model=MODEL_NAME,
    messages=[
        {"role": "user", "content": prompt_text}
    ],
    tools=tools,
    tool_choice={
        "type": "function",
        "function": {"name": "get_products"}
    },
    response_format={"type": "json_object"}
)

# ツール呼び出し情報
tool = response.choices[0].message.tool_calls[0]

# jsonオブジェクト変換
data = json.loads(tool.function.arguments)

# データフレーム変換
df_out = pd.DataFrame(data['product_data'])

# 結果を保存
df_out.to_excel("商品名グループ化結果.xlsx", index=False)

print("Excelファイルに処理結果を保存しました。")

処理を開始します。
Excelファイルに処理結果を保存しました。
